# OAI X-ray Inpainting - Google Colab

This notebook provides a complete environment for training and testing inpainting models on the OAI dataset.

## 🚀 Quick Start

1. **Setup Environment** - Run the setup cell below
2. **Upload Data** - Upload your OAI dataset files
3. **Train Models** - Run training for AOT-GAN, ICT, or RePaint
4. **Test Models** - Evaluate model performance
5. **Download Results** - Save results to your local machine

---


## 🔧 Environment Setup


In [ ]:
# Check GPU availability
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

In [ ]:
# Clone repository and install dependencies
import os
import subprocess
from pathlib import Path

# Clone the repository
if not Path("OAI-inpainting").exists():
    print("📥 Cloning repository...")
    subprocess.run(
        ["git", "clone", "https://github.com/johnreynolds3d/OAI-inpainting.git"],
        check=True,
    )
else:
    print("📁 Repository already exists, updating...")
    os.chdir("OAI-inpainting")
    subprocess.run(["git", "pull"], check=True)
    os.chdir("..")

# Change to project directory
os.chdir("OAI-inpainting")
print(f"📂 Current directory: {Path.cwd()}")

# Install dependencies
print("📦 Installing dependencies...")
subprocess.run(["pip", "install", "-e", ".[dev,ml]"], check=True)

print("✅ Setup complete!")

In [ ]:
# Verify installation
try:
    from src.paths import get_project_root

    print("✅ Core modules imported successfully")
    print(f"📁 Project root: {get_project_root()}")
except ImportError as e:
    print(f"❌ Import error: {e}")

# Check available models
models_dir = Path("models")
if models_dir.exists():
    print("\n📋 Available models:")
    for model in models_dir.iterdir():
        if model.is_dir():
            print(f"  - {model.name}")
else:
    print("❌ Models directory not found")

## 🔄 Update Repository


In [ ]:
# Update repository to latest version
print("🔄 Updating repository...")

try:
    # Fetch latest changes
    subprocess.run(["git", "fetch"], check=True)

    # Check for updates
    result = subprocess.run(
        ["git", "status", "-uno"], check=False, capture_output=True, text=True
    )
    print("📋 Repository status:")
    print(result.stdout)

    # Pull latest changes
    subprocess.run(["git", "pull"], check=True)
    print("✅ Repository updated to latest version")

    # Reinstall dependencies if needed
    print("📦 Reinstalling dependencies...")
    subprocess.run(["pip", "install", "-e", ".[dev,ml]"], check=True)
    print("✅ Dependencies updated")

except Exception as e:
    print(f"❌ Update failed: {e}")
    print("💡 You may need to restart the runtime and re-run the setup cell.")